Prior to loading the dataset into R, it was pre-processed in Excel. The raw elemental intensities were each divided by the Rh column, which scaled each element, and a new column for each element was generated. Then, all amounts less than or equal to 0 were replaced with 0.0001, to avoid any issues with NaNs in the dataset.

# Load libraries

In [1]:
# try installing vctrs fresh, since it is giving me errors
#install.packages("vctrs", dependencies = TRUE, repos = 'http://cran.us.r-project.org')
library(vctrs)

# Dicer library is used to run ensemble clustering, which generates interim labels for unknown samples
#install.packages("diceR", dependencies = TRUE, repos = 'http://cran.us.r-project.org')
library(diceR)

# try installing Biobase, since I got error messages
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install("Biobase")
library(Biobase)

# randomForest is the classification system which identifies unknown samples
#install.packages("randomForest", dependencies = TRUE, repos = 'http://cran.us.r-project.org')
library(randomForest)

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To 

# Load train dataset

In [2]:
train <- read.csv("AllSamples.csv", header = TRUE)
head(train)

,X,is_known,Vis,Ag,Al,As,Au,Ca,Cu,Fe,⋯,Si.Rh,Sn.Rh,Sr.Rh,Th.Rh,Ti.Rh,U.Rh,V.Rh,Y.Rh,Zn.Rh,Zr.Rh
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,71.11.1-89.01,Guess,Gravel Cherts,1368,1,0.0001,54,1515,331,5335,⋯,0.06234530,0.030244431,0.13691213,2.328280e-02,0.015702351,0.027769183,0.005491955,0.003635520,0.090423886,7.740000e-09
2,71.11.1-89.02,Guess,Gravel Cherts,1269,15,55.0000,25,682,300,2670,⋯,0.06373444,0.019170124,0.18995851,1.294606e-02,0.013609959,0.016431535,0.006887967,0.000083000,0.071867220,8.300000e-09
3,71.11.1-89.03,Guess,Gravel Cherts,1165,32,4.0000,46,607,523,6119,⋯,0.05545573,0.010944136,0.09882391,7.350539e-03,0.018703038,0.008330611,0.005308723,0.004573669,0.039692911,2.531852e-03
4,71.11.1-89.04,Guess,Gravel Cherts,1299,40,42.0000,45,338,410,5307,⋯,0.06011742,0.010667328,0.04589432,1.174233e-02,0.021582734,0.024559663,0.001901927,0.011080791,0.032746217,4.796163e-03
5,71.11.1-89.05,Guess,Gravel Cherts,1724,35,0.0001,51,201,301,1745,⋯,0.07111721,0.002502422,0.08411366,6.780756e-03,0.010251857,0.010494026,0.005247013,0.012431385,0.009283177,8.070000e-09
6,71.11.1-89.06,Guess,Gravel Cherts,1246,139,159.0000,18,188,292,4858,⋯,0.07108405,0.009646556,0.09575393,7.910000e-09,0.008381434,0.006167471,0.001897683,0.000079100,0.014627975,5.851190e-03


In [3]:
# restrict dataset to only important columns: sample id, is_known, and the elemental intensities
train_df = train[,c(1,2,3,32:58)]
# rename X to sample_id
names(train_df)[1] <- "sample_id"
# preview dataset
head(train_df)

,sample_id,is_known,Vis,Ag.Rh,Al.Rh,As.Rh,Au.Rh,Ca.Rh,Cu.Rh,Fe.Rh,⋯,Si.Rh,Sn.Rh,Sr.Rh,Th.Rh,Ti.Rh,U.Rh,V.Rh,Y.Rh,Zn.Rh,Zr.Rh
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,71.11.1-89.01,Guess,Gravel Cherts,0.10581683,0.000077400,7.740000e-09,0.004176980,0.11718750,0.02560334,0.4126702,⋯,0.06234530,0.030244431,0.13691213,2.328280e-02,0.015702351,0.027769183,0.005491955,0.003635520,0.090423886,7.740000e-09
2,71.11.1-89.02,Guess,Gravel Cherts,0.10531120,0.001244813,4.564315e-03,0.002074689,0.05659751,0.02489627,0.2215768,⋯,0.06373444,0.019170124,0.18995851,1.294606e-02,0.013609959,0.016431535,0.006887967,0.000083000,0.071867220,8.300000e-09
3,71.11.1-89.03,Guess,Gravel Cherts,0.09514864,0.002613525,3.266910e-04,0.003756942,0.04957530,0.04271480,0.4997550,⋯,0.05545573,0.010944136,0.09882391,7.350539e-03,0.018703038,0.008330611,0.005308723,0.004573669,0.039692911,2.531852e-03
4,71.11.1-89.04,Guess,Gravel Cherts,0.10741751,0.003307699,3.473084e-03,0.003721161,0.02795005,0.03390391,0.4388489,⋯,0.06011742,0.010667328,0.04589432,1.174233e-02,0.021582734,0.024559663,0.001901927,0.011080791,0.032746217,4.796163e-03
5,71.11.1-89.05,Guess,Gravel Cherts,0.13916694,0.002825315,8.070000e-09,0.004116887,0.01622538,0.02429771,0.1408621,⋯,0.07111721,0.002502422,0.08411366,6.780756e-03,0.010251857,0.010494026,0.005247013,0.012431385,0.009283177,8.070000e-09
6,71.11.1-89.06,Guess,Gravel Cherts,0.09852139,0.010990749,1.257215e-02,0.001423262,0.01486518,0.02308848,0.3841227,⋯,0.07108405,0.009646556,0.09575393,7.910000e-09,0.008381434,0.006167471,0.001897683,0.000079100,0.014627975,5.851190e-03


# Run ensemble clustering on train dataset

For this portion we used diceR to run a consensus cluster with the following settings:
- nk = 2 - 15 clusters, to be sure I captured all possible groupings
- reps = 1 because I found that multiple attempts to cluster the same way didn't change the result significantly
- We also broke up the number of clusters to alleviate processing time, so a max of 4 algorithms are run in each command.

In [4]:
cluster1 <- consensus_cluster(train_df[,4:30], nk=2:15, p.item=1, reps=1, 
                             algorithms=c("nmf", "hc", "diana", "km"), scale = FALSE)

New names:
* Ag.Rh -> Ag.Rh...1
* Al.Rh -> Al.Rh...2
* As.Rh -> As.Rh...3
* Au.Rh -> Au.Rh...4
* Ca.Rh -> Ca.Rh...5
* ...


Clustering Algorithm 1 of 5: NMF_Brunet (k = 3) [--------------]   3% eta:  4m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 4) [>-------------]   4% eta: 10m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 5) [>-------------]   6% eta: 12m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 6) [>-------------]   7% eta: 11m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 7) [>-------------]   9% eta: 13m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 8) [>-------------]  10% eta: 13m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 9) [=>------------]  11% eta: 14m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 10) [=>-----------]  13% eta: 14m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 11) [=>-----------]  14% eta: 14m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 12) [=>-----------]  16% eta: 15m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 13) [=>-----------]  17% eta: 1

In [5]:
cluster2 <- consensus_cluster(train_df[,4:30], nk=2:15, p.item=1, reps=1, 
                             algorithms=c("pam", "ap", "sc", "gmm"), scale = FALSE)


Clustering Algorithm 1 of 4: PAM_Euclidean (k = 4) [>----------]   5% eta:  6s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 5) [>----------]   7% eta:  9s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 6) [>----------]   9% eta: 10s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 7) [>----------]  11% eta: 10s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 8) [>----------]  12% eta: 12s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 9) [=>---------]  14% eta: 14s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 10) [=>--------]  16% eta: 16s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 11) [=>--------]  18% eta: 18s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 12) [=>--------]  20% eta: 20s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 13) [=>--------]  21% eta: 24s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 14) [=>--------]  23% eta: 27s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 15) [=>--------]  25% eta: 29s

Clustering Algorithm 2 of 4: AP (k = 2)

In [6]:
cluster3 <- consensus_cluster(train_df[,4:30], nk=2:15, p.item=1, reps=1, 
                             algorithms=c("block", "som", "cmeans"), scale = FALSE)


Clustering Algorithm 1 of 3: BLOCK (k = 3) [>------------------]   5% eta:  4s

Clustering Algorithm 1 of 3: BLOCK (k = 4) [>------------------]   7% eta:  7s

Clustering Algorithm 1 of 3: BLOCK (k = 5) [=>-----------------]  10% eta:  8s

Clustering Algorithm 1 of 3: BLOCK (k = 6) [=>-----------------]  12% eta: 10s

Clustering Algorithm 1 of 3: BLOCK (k = 7) [==>----------------]  14% eta: 16s

Clustering Algorithm 1 of 3: BLOCK (k = 8) [==>----------------]  17% eta: 21s

Clustering Algorithm 1 of 3: BLOCK (k = 9) [===>---------------]  19% eta: 31s

Clustering Algorithm 1 of 3: BLOCK (k = 10) [===>--------------]  21% eta: 41s

Clustering Algorithm 1 of 3: BLOCK (k = 11) [===>--------------]  24% eta:  1m

Clustering Algorithm 1 of 3: BLOCK (k = 12) [====>-------------]  26% eta:  1m

Clustering Algorithm 1 of 3: BLOCK (k = 13) [====>-------------]  29% eta:  1m

Clustering Algorithm 1 of 3: BLOCK (k = 14) [=====>------------]  31% eta:  1m

Clustering Algorithm 1 of 3: BLOCK (k =

# Choose a clustering method

In this section we examine each of the results of the clustering algorithms and choose that which best groups our 'known' groups together and separates them out. Before this can be done, however, we need to do some work turning the results of the clustering step into readable data frames.

In [7]:
# save the results of the algorithm as individual dataframes
brunet <- cluster1[,,"NMF_Brunet",1:14]
lee <- cluster1[,,"NMF_Lee",1:14]
hc <- cluster1[,,"HC_Euclidean",1:14]
diana <- cluster1[,,"DIANA_Euclidean",1:14]
km <- cluster1[,,"KM",1:14]
pam <- cluster2[,,"PAM_Euclidean",1:14]
ap <- cluster2[,,"AP",1:14]
sc <- cluster2[,,"SC",1:14]
gmm <- cluster2[,,"GMM",1:14]
block <- cluster3[,,"BLOCK",1:14]
som <- cluster3[,,"SOM",1:14]
cmeans <- cluster3[,,"CMEANS",1:14]

In [8]:
# preview one of the tables
head(brunet)

,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,2,1,1,5,5,5,5,5,1,1,5,1,1,12
2,2,1,1,5,5,5,5,5,1,1,1,2,9,1
3,2,1,1,5,5,5,5,8,1,1,1,1,1,1
4,2,1,1,5,5,5,5,5,1,1,1,2,9,1
5,2,1,1,5,5,5,5,5,1,1,1,2,9,1
6,2,1,1,5,5,5,5,5,1,1,1,2,9,1


In [9]:
# Each number in the table refers to the group that each sample has been assigned to.
# Edit each value in the tables by adding "Group_" to each of the numbers so that they are strings and can be counted, not summed, by crosstab
brunet[,1:14] <- paste("Group", brunet[,1:14], sep = "_")
lee[,1:14] <- paste("Group", lee[,1:14], sep = "_")
hc[,1:14] <- paste("Group", hc[,1:14], sep = "_")
diana[,1:14] <- paste("Group", diana[,1:14], sep = "_")
km[,1:14] <- paste("Group", km[,1:14], sep = "_")
pam[,1:14] <- paste("Group", pam[,1:14], sep = "_")
ap[,1:14] <- paste("Group", ap[,1:14], sep = "_")
sc[,1:14] <- paste("Group", sc[,1:14], sep = "_")
gmm[,1:14] <- paste("Group", gmm[,1:14], sep = "_")
block[,1:14] <- paste("Group", block[,1:14], sep = "_")
som[,1:14] <- paste("Group", som[,1:14], sep = "_")
cmeans[,1:14] <- paste("Group", cmeans[,1:14], sep = "_")
head(brunet)

,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_5,Group_1,Group_1,Group_12
2,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
3,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_8,Group_1,Group_1,Group_1,Group_1,Group_1,Group_1
4,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
5,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
6,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1


In [10]:
# turn each of these matrices into dataframes
brunet <- as.data.frame(brunet)
lee <- as.data.frame(lee)
hc <- as.data.frame(hc)
diana <- as.data.frame(diana)
km <- as.data.frame(km)
pam <- as.data.frame(pam)
ap <- as.data.frame(ap)
sc <- as.data.frame(sc)
gmm <- as.data.frame(gmm)
block <- as.data.frame(block)
som <- as.data.frame(som)
cmeans <- as.data.frame(cmeans)

head(brunet)

,2,3,4,5,6,7,8,9,10,11,12,13,14,15
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_5,Group_1,Group_1,Group_12
2,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
3,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_8,Group_1,Group_1,Group_1,Group_1,Group_1,Group_1
4,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
5,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
6,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1


In [11]:
# then add the is_known & vis columns back in to each df
brunet$is_known <- train$is_known
brunet$vis <- train$Vis

lee$is_known <- train$is_known
lee$vis <- train$Vis

hc$is_known <- train$is_known
hc$vis <- train$Vis

diana$is_known <- train$is_known
diana$vis <- train$Vis

km$is_known <- train$is_known
km$vis <- train$Vis

pam$is_known <- train$is_known
pam$vis <- train$Vis

ap$is_known <- train$is_known
ap$vis <- train$Vis

sc$is_known <- train$is_known
sc$vis <- train$Vis

gmm$is_known <- train$is_known
gmm$vis <- train$Vis

block$is_known <- train$is_known
block$vis <- train$Vis

som$is_known <- train$is_known
som$vis <- train$Vis

cmeans$is_known <- train$is_known
cmeans$vis <- train$Vis

head(brunet)

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,is_known,vis
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_5,Group_1,Group_1,Group_12,Guess,Gravel Cherts
2,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1,Guess,Gravel Cherts
3,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_8,Group_1,Group_1,Group_1,Group_1,Group_1,Group_1,Guess,Gravel Cherts
4,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1,Guess,Gravel Cherts
5,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1,Guess,Gravel Cherts
6,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1,Guess,Gravel Cherts


# Algorithm Selection Pt 2

In the previous notebook, we ran the clustering algorithms and checked their results. In this notebook, we will narrow down the results to only those which perform well, and check the results on artifacts to determine group labels. 

In [182]:
# Set the max rows options to something bigger for this section
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)

In [77]:
# not good enough...
# this one splits up oman & black cherts into chert_1 & chert_2
# and puts silicified wood & limestone into chert_2

# Group 1 = 
# Group 2 = Chert_1
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = Chert_2
# Group 7 = 
# Group 8 = ESC
# Group 9 = 
# Group 10 = 

km_10_raw <- km[km$is_known == "Known",c(9,16)]
km_10_raw <- ftable(km_10_raw[])
km_10_raw <- as.data.frame(km_10_raw)
km_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_2,Alibates,20
Group_3,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,3
Group_7,Alibates,0
Group_8,Alibates,0
Group_10,Edwards Plateau Chert,1
Group_2,Edwards Plateau Chert,56


In [185]:
km_10_art <- km[km$is_known != "Known",c(9,16)]
km_10_art <- ftable(km_10_art[])
km_10_art <- as.data.frame(km_10_art)
km_10_art

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Agate,0
Group_10,Agate,0
Group_2,Agate,47
Group_3,Agate,0
Group_4,Agate,0
Group_5,Agate,0
Group_6,Agate,2
Group_7,Agate,0
Group_8,Agate,1


In [78]:
# not good enough...
# this one puts a variety of other types into the oman group

# Group 1 = 
# Group 2 = Chert_1
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = Oman
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = ESC

km_11_raw <- km[km$is_known == "Known",c(10,16)]
km_11_raw <- ftable(km_11_raw[])
km_11_raw <- as.data.frame(km_11_raw)
km_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_2,Alibates,18
Group_3,Alibates,0
Group_5,Alibates,1
Group_6,Alibates,4
Group_7,Alibates,0
Group_8,Alibates,0
Group_10,Edwards Plateau Chert,1


In [186]:
km_11_art <- km[km$is_known != "Known",c(10,16)]
km_11_art <- ftable(km_11_art[])
km_11_art <- as.data.frame(km_11_art)
km_11_art

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Agate,0
Group_10,Agate,0
Group_11,Agate,0
Group_2,Agate,46
Group_3,Agate,0
Group_4,Agate,0
Group_5,Agate,1
Group_6,Agate,2
Group_7,Agate,0


In [79]:
# not good enough...
# this one puts a variety of other types in with the oman group

# Group 1 = 
# Group 2 = Chert_1
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = Oman
# Group 7 = Silicified_wood
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = ESC
# Group 12 = 

km_12_raw <- km[km$is_known == "Known",c(11,16)]
km_12_raw <- ftable(km_12_raw[])
km_12_raw <- as.data.frame(km_12_raw)
km_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,17
Group_4,Alibates,0
Group_6,Alibates,5
Group_7,Alibates,1
Group_8,Alibates,0
Group_10,Edwards Plateau Chert,1


In [187]:
km_12_art <- km[km$is_known != "Known",c(11,16)]
km_12_art <- ftable(km_12_art[])
km_12_art <- as.data.frame(km_12_art)
km_12_art

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Agate,0
Group_10,Agate,0
Group_11,Agate,0
Group_12,Agate,0
Group_2,Agate,46
Group_3,Agate,0
Group_4,Agate,0
Group_5,Agate,0
Group_6,Agate,2


In [80]:
# not good enough...
# a variety of types are mxed with oman

# Group 1 = 
# Group 2 = Chert_1
# Group 3 = 
# Group 4 = 
# Group 5 = Oman
# Group 6 = Chert_2
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = ESC
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

km_13_raw <- km[km$is_known == "Known",c(12,16)]
km_13_raw <- ftable(km_13_raw[])
km_13_raw <- as.data.frame(km_13_raw)
km_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,15
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,7
Group_7,Alibates,1
Group_8,Alibates,0


In [188]:
km_13_art <- km[km$is_known != "Known",c(12,16)]
km_13_art <- ftable(km_13_art[])
km_13_art <- as.data.frame(km_13_art)
km_13_art

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Agate,0
Group_10,Agate,0
Group_11,Agate,0
Group_12,Agate,0
Group_13,Agate,0
Group_2,Agate,45
Group_3,Agate,0
Group_4,Agate,0
Group_5,Agate,0


In [81]:
# this one is better, but 34 ESC are still with the rest
# ESC = 187, Oman = 11

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

km_14_raw <- km[km$is_known == "Known",c(13,16)]
km_14_raw <- ftable(km_14_raw[])
km_14_raw <- as.data.frame(km_14_raw)
km_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,14
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,7
Group_7,Alibates,2


In [82]:
# this one is better, but 36 ESC are still with the rest
# ESC = 185, Oman = 11

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

km_15_raw <- km[km$is_known == "Known",c(14,16)]
km_15_raw <- ftable(km_15_raw[])
km_15_raw <- as.data.frame(km_15_raw)
km_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_14,Alibates,0
Group_15,Alibates,4
Group_2,Alibates,14
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,4


In [88]:
# this one puts 16 ESC with the rest, but it sorts out oman better than the previous option
# ESC = 235, Oman = 17

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

pam_7_raw <- pam[pam$is_known == "Known",c(6,16)]
pam_7_raw <- ftable(pam_7_raw[])
pam_7_raw <- as.data.frame(pam_7_raw)
pam_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,8
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,2
Group_4,Edwards Plateau Chert,19


In [89]:
# this one puts 16 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 235, Oman = 19

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

pam_8_raw <- pam[pam$is_known == "Known",c(7,16)]
pam_8_raw <- ftable(pam_8_raw[])
pam_8_raw <- as.data.frame(pam_8_raw)
pam_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,7
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,1


In [90]:
# this one puts 16 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 235, Oman = 19

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

pam_9_raw <- pam[pam$is_known == "Known",c(8,16)]
pam_9_raw <- ftable(pam_9_raw[])
pam_9_raw <- as.data.frame(pam_9_raw)
pam_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,7
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,1


In [91]:
# this one puts 16 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 235, Oman = 19

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

pam_10_raw <- pam[pam$is_known == "Known",c(9,16)]
pam_10_raw <- ftable(pam_10_raw[])
pam_10_raw <- as.data.frame(pam_10_raw)
pam_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,7
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,1


In [92]:
# this one puts 16 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 211, Oman = 19

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

pam_11_raw <- pam[pam$is_known == "Known",c(10,16)]
pam_11_raw <- ftable(pam_11_raw[])
pam_11_raw <- as.data.frame(pam_11_raw)
pam_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,1
Group_6,Alibates,7
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45


In [93]:
# this one puts 14 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 213, Oman = 17

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

pam_12_raw <- pam[pam$is_known == "Known",c(11,16)]
pam_12_raw <- ftable(pam_12_raw[])
pam_12_raw <- as.data.frame(pam_12_raw)
pam_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_2,Alibates,14
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,6
Group_6,Alibates,2
Group_8,Alibates,1
Group_1,Edwards Plateau Chert,0


In [94]:
# this one puts 17 ESC with the rest, and it sorts out oman better than the previous few options
# ESC = 205, Oman = 13

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

pam_13_raw <- pam[pam$is_known == "Known",c(12,16)]
pam_13_raw <- ftable(pam_13_raw[])
pam_13_raw <- as.data.frame(pam_13_raw)
pam_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_11,Alibates,0
Group_2,Alibates,12
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,6
Group_6,Alibates,4
Group_7,Alibates,0
Group_9,Alibates,1


In [95]:
# this one puts 36 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 205, Oman = 13

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

pam_14_raw <- pam[pam$is_known == "Known",c(13,16)]
pam_14_raw <- ftable(pam_14_raw[])
pam_14_raw <- as.data.frame(pam_14_raw)
pam_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,12
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,3
Group_6,Alibates,1
Group_7,Alibates,0


In [96]:
# this one puts 36 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 205, Oman = 13

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

pam_15_raw <- pam[pam$is_known == "Known",c(14,16)]
pam_15_raw <- ftable(pam_15_raw[])
pam_15_raw <- as.data.frame(pam_15_raw)
pam_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,12
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,3
Group_6,Alibates,1
Group_7,Alibates,0


In [101]:
# this one is good, it separates both ESC and most of Oman
# ESC = 247, Oman = 15

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_6_raw <- ap[ap$is_known == "Known",c(5,16)]
ap_6_raw <- ftable(ap_6_raw[])
ap_6_raw <- as.data.frame(ap_6_raw)
ap_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,5
Group_3,Alibates,18
Group_4,Alibates,0
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,14
Group_3,Edwards Plateau Chert,50
Group_4,Edwards Plateau Chert,6


In [102]:
# this one is good, it separates both ESC and most of Oman
# ESC = 248, Oman = 14

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_7_raw <- ap[ap$is_known == "Known",c(6,16)]
ap_7_raw <- ftable(ap_7_raw[])
ap_7_raw <- as.data.frame(ap_7_raw)
ap_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,20
Group_6,Alibates,3
Group_1,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,5
Group_4,Edwards Plateau Chert,1
Group_5,Edwards Plateau Chert,53


In [103]:
# this one is good, it separates both ESC and most of Oman
# ESC = 240, Oman = 14

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_8_raw <- ap[ap$is_known == "Known",c(7,16)]
ap_8_raw <- ftable(ap_8_raw[])
ap_8_raw <- as.data.frame(ap_8_raw)
ap_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,0
Group_6,Alibates,7
Group_7,Alibates,0
Group_8,Alibates,14
Group_1,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,9
Group_4,Edwards Plateau Chert,5


In [104]:
# this one is good, it separates both ESC and most of Oman
# ESC = 235, Oman = 18

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_9_raw <- ap[ap$is_known == "Known",c(8,16)]
ap_9_raw <- ftable(ap_9_raw[])
ap_9_raw <- as.data.frame(ap_9_raw)
ap_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,7
Group_2,Alibates,0
Group_4,Alibates,0
Group_6,Alibates,15
Group_7,Alibates,0
Group_9,Alibates,1
Group_1,Edwards Plateau Chert,17
Group_2,Edwards Plateau Chert,0
Group_4,Edwards Plateau Chert,2


In [105]:
# this one is good, it separates both ESC and most of Oman
# ESC = 235, Oman = 17

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_10_raw <- ap[ap$is_known == "Known",c(9,16)]
ap_10_raw <- ftable(ap_10_raw[])
ap_10_raw <- as.data.frame(ap_10_raw)
ap_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,7
Group_2,Alibates,0
Group_3,Alibates,1
Group_5,Alibates,0
Group_7,Alibates,0
Group_8,Alibates,15
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,17
Group_2,Edwards Plateau Chert,0


In [106]:
# this one is good, it separates both ESC and most of Oman
# ESC = 235, Oman = 17

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_11_raw <- ap[ap$is_known == "Known",c(10,16)]
ap_11_raw <- ftable(ap_11_raw[])
ap_11_raw <- as.data.frame(ap_11_raw)
ap_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,7
Group_2,Alibates,0
Group_3,Alibates,1
Group_5,Alibates,0
Group_7,Alibates,0
Group_8,Alibates,15
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,17
Group_2,Edwards Plateau Chert,0


In [107]:
# this one is good, it separates both ESC and most of Oman
# ESC = 194 & 50, Oman = 16

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_12_raw <- ap[ap$is_known == "Known",c(11,16)]
ap_12_raw <- ftable(ap_12_raw[])
ap_12_raw <- as.data.frame(ap_12_raw)
ap_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_10,Alibates,0
Group_11,Alibates,14
Group_3,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,0
Group_9,Alibates,8
Group_1,Edwards Plateau Chert,5


In [108]:
# this one is good, it separates both ESC and most of Oman
# ESC = 205, Oman = 13

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_13_raw <- ap[ap$is_known == "Known",c(12,16)]
ap_13_raw <- ftable(ap_13_raw[])
ap_13_raw <- as.data.frame(ap_13_raw)
ap_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,3
Group_12,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,12
Group_4,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,0


In [109]:
# this one is good, it separates both ESC and most of Oman
# ESC = 205, Oman = 13

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_14_raw <- ap[ap$is_known == "Known",c(13,16)]
ap_14_raw <- ftable(ap_14_raw[])
ap_14_raw <- as.data.frame(ap_14_raw)
ap_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,3
Group_12,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,12
Group_4,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,0


In [110]:
# this one is good, it separates both ESC and most of Oman
# ESC = 205, Oman = 13

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

ap_15_raw <- ap[ap$is_known == "Known",c(14,16)]
ap_15_raw <- ftable(ap_15_raw[])
ap_15_raw <- as.data.frame(ap_15_raw)
ap_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,3
Group_12,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,12
Group_4,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,0


In [115]:
# this one is good, it separates both ESC and most of Oman
# ESC = 159, Oman = 15

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

sc_6_raw <- sc[sc$is_known == "Known",c(5,16)]
sc_6_raw <- ftable(sc_6_raw[])
sc_6_raw <- as.data.frame(sc_6_raw)
sc_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,5
Group_2,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,16
Group_1,Edwards Plateau Chert,18
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,11


In [116]:
# this one is good, it separates both ESC and most of Oman
# ESC = 158, Oman = 21

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

sc_7_raw <- sc[sc$is_known == "Known",c(6,16)]
sc_7_raw <- ftable(sc_7_raw[])
sc_7_raw <- as.data.frame(sc_7_raw)
sc_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,4
Group_3,Alibates,17
Group_4,Alibates,1
Group_5,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,1
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,18


In [117]:
# this one is good, it separates both ESC and most of Oman
# ESC = 118 & 90, Oman = 18

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

sc_8_raw <- sc[sc$is_known == "Known",c(7,16)]
sc_8_raw <- ftable(sc_8_raw[])
sc_8_raw <- as.data.frame(sc_8_raw)
sc_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,6
Group_2,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,13
Group_5,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,1
Group_8,Alibates,1
Group_1,Edwards Plateau Chert,15


In [118]:
# this one is good, it separates both ESC and most of Oman
# ESC = 119 & 83, Oman = 17

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

sc_9_raw <- sc[sc$is_known == "Known",c(8,16)]
sc_9_raw <- ftable(sc_9_raw[])
sc_9_raw <- as.data.frame(sc_9_raw)
sc_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,1
Group_4,Alibates,1
Group_5,Alibates,2
Group_6,Alibates,0
Group_7,Alibates,0
Group_8,Alibates,13
Group_9,Alibates,6


In [119]:
# this one is good, it separates both ESC and most of Oman
# ESC = 110 & 81, Oman = 21

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

sc_10_raw <- sc[sc$is_known == "Known",c(9,16)]
sc_10_raw <- ftable(sc_10_raw[])
sc_10_raw <- as.data.frame(sc_10_raw)
sc_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,12
Group_2,Alibates,0
Group_3,Alibates,3
Group_4,Alibates,1
Group_5,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,1
Group_8,Alibates,0


In [120]:
# this one is good, it separates both ESC and most of Oman
# ESC = 99 & 62 & 66, Oman = 21

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

sc_11_raw <- sc[sc$is_known == "Known",c(10,16)]
sc_11_raw <- ftable(sc_11_raw[])
sc_11_raw <- as.data.frame(sc_11_raw)
sc_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,1
Group_11,Alibates,1
Group_2,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,8
Group_5,Alibates,0
Group_6,Alibates,4
Group_7,Alibates,1


In [121]:
# this one is good, it separates both ESC and most of Oman
# ESC = 105, 63, 66, Oman = 21

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

sc_12_raw <- sc[sc$is_known == "Known",c(11,16)]
sc_12_raw <- ftable(sc_12_raw[])
sc_12_raw <- as.data.frame(sc_12_raw)
sc_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,5
Group_11,Alibates,0
Group_12,Alibates,8
Group_2,Alibates,1
Group_3,Alibates,0
Group_4,Alibates,6
Group_5,Alibates,0
Group_6,Alibates,0


In [127]:
# this one is good, it separates both ESC and most of Oman
# ESC = 226, Oman = 18

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_4_raw <- gmm[gmm$is_known == "Known",c(3,16)]
gmm_4_raw <- ftable(gmm_4_raw[])
gmm_4_raw <- as.data.frame(gmm_4_raw)
gmm_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,3
Group_2,Alibates,16
Group_3,Alibates,0
Group_4,Alibates,4
Group_1,Edwards Plateau Chert,1
Group_2,Edwards Plateau Chert,50
Group_3,Edwards Plateau Chert,0
Group_4,Edwards Plateau Chert,21
Group_1,El Sauz Chert,226


In [128]:
# this one is good, it separates both ESC and most of Oman
# ESC = 236, Oman = 13

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_5_raw <- gmm[gmm$is_known == "Known",c(4,16)]
gmm_5_raw <- ftable(gmm_5_raw[])
gmm_5_raw <- as.data.frame(gmm_5_raw)
gmm_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,5
Group_2,Alibates,16
Group_3,Alibates,0
Group_4,Alibates,2
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,21
Group_2,Edwards Plateau Chert,46
Group_3,Edwards Plateau Chert,0
Group_4,Edwards Plateau Chert,0


In [129]:
# this one is good, it separates both ESC and most of Oman
# ESC = 178 & 93, Oman = 12

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_6_raw <- gmm[gmm$is_known == "Known",c(5,16)]
gmm_6_raw <- ftable(gmm_6_raw[])
gmm_6_raw <- as.data.frame(gmm_6_raw)
gmm_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,5
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,2
Group_5,Alibates,16
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,22
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,0


In [130]:
# this one is good, it separates both ESC and most of Oman
# ESC = 189 & 67, Oman = 11

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_7_raw <- gmm[gmm$is_known == "Known",c(6,16)]
gmm_7_raw <- ftable(gmm_7_raw[])
gmm_7_raw <- as.data.frame(gmm_7_raw)
gmm_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,4
Group_2,Alibates,9
Group_3,Alibates,2
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,8
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,12
Group_2,Edwards Plateau Chert,26


In [131]:
# this one is good, it separates both ESC and most of Oman
# ESC = 205, Oman = 9

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_8_raw <- gmm[gmm$is_known == "Known",c(7,16)]
gmm_8_raw <- ftable(gmm_8_raw[])
gmm_8_raw <- as.data.frame(gmm_8_raw)
gmm_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_2,Alibates,8
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,4
Group_6,Alibates,3
Group_7,Alibates,7
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,8


In [132]:
# this one is good, it separates both ESC and most of Oman
# ESC = 206, Oman = 9

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_9_raw <- gmm[gmm$is_known == "Known",c(8,16)]
gmm_9_raw <- ftable(gmm_9_raw[])
gmm_9_raw <- as.data.frame(gmm_9_raw)
gmm_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_2,Alibates,3
Group_3,Alibates,3
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,11
Group_7,Alibates,1
Group_8,Alibates,4
Group_9,Alibates,0


In [133]:
# this one is good, it separates both ESC and most of Oman
# ESC = 106 & 119, Oman = 9

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_10_raw <- gmm[gmm$is_known == "Known",c(9,16)]
gmm_10_raw <- ftable(gmm_10_raw[])
gmm_10_raw <- as.data.frame(gmm_10_raw)
gmm_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,4
Group_10,Alibates,0
Group_2,Alibates,8
Group_3,Alibates,2
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,3
Group_7,Alibates,3
Group_8,Alibates,0


In [134]:
# this one is good, it separates both ESC and most of Oman
# ESC = 129, 63 & 55, Oman = 9 & 9

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_11_raw <- gmm[gmm$is_known == "Known",c(10,16)]
gmm_11_raw <- ftable(gmm_11_raw[])
gmm_11_raw <- as.data.frame(gmm_11_raw)
gmm_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,4
Group_11,Alibates,0
Group_2,Alibates,5
Group_3,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,0
Group_6,Alibates,1
Group_7,Alibates,7


In [135]:
# this one is good, it separates both ESC and most of Oman
# ESC = 115 & 103, Oman = 9 & 9

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_12_raw <- gmm[gmm$is_known == "Known",c(11,16)]
gmm_12_raw <- ftable(gmm_12_raw[])
gmm_12_raw <- as.data.frame(gmm_12_raw)
gmm_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,4
Group_11,Alibates,0
Group_2,Alibates,3
Group_3,Alibates,2
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,1
Group_7,Alibates,9


In [136]:
# this one is good, it separates both ESC and most of Oman
# ESC = 104 & 111, Oman = 9 & 9

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

gmm_13_raw <- gmm[gmm$is_known == "Known",c(12,16)]
gmm_13_raw <- ftable(gmm_13_raw[])
gmm_13_raw <- as.data.frame(gmm_13_raw)
gmm_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,4
Group_11,Alibates,3
Group_12,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,5
Group_5,Alibates,0
Group_6,Alibates,1


In [142]:
# this one is good, it separates both ESC and most of Oman
# ESC = 223, Oman = 11

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

block_5_raw <- block[block$is_known == "Known",c(4,16)]
block_5_raw <- ftable(block_5_raw[])
block_5_raw <- as.data.frame(block_5_raw)
block_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,14
Group_3,Alibates,0
Group_4,Alibates,6
Group_5,Alibates,3
Group_1,Edwards Plateau Chert,1
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,3
Group_4,Edwards Plateau Chert,9


In [144]:
# this one is good, it separates both ESC and most of Oman
# ESC = 184, Oman = 11

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

block_7_raw <- block[block$is_known == "Known",c(6,16)]
block_7_raw <- ftable(block_7_raw[])
block_7_raw <- as.data.frame(block_7_raw)
block_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,2
Group_2,Alibates,0
Group_3,Alibates,14
Group_4,Alibates,2
Group_5,Alibates,3
Group_6,Alibates,2
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,2
Group_2,Edwards Plateau Chert,0


In [145]:
# this one is good, it separates both ESC and most of Oman
# ESC = 170 & 91, Oman = 12

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

block_8_raw <- block[block$is_known == "Known",c(7,16)]
block_8_raw <- ftable(block_8_raw[])
block_8_raw <- as.data.frame(block_8_raw)
block_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,3
Group_2,Alibates,2
Group_3,Alibates,1
Group_4,Alibates,14
Group_5,Alibates,1
Group_6,Alibates,2
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,9
Group_2,Edwards Plateau Chert,14


In [147]:
# this one is good, it separates both ESC and most of Oman
# ESC = 203, Oman = 11

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

block_10_raw <- block[block$is_known == "Known",c(9,16)]
block_10_raw <- ftable(block_10_raw[])
block_10_raw <- as.data.frame(block_10_raw)
block_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,1
Group_2,Alibates,0
Group_3,Alibates,14
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,2
Group_7,Alibates,1
Group_8,Alibates,3


In [175]:
# this one is good, it separates both ESC and most of Oman
# ESC = 204, Oman = 13

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

cmeans_10_raw <- cmeans[cmeans$is_known == "Known",c(9,16)]
cmeans_10_raw <- ftable(cmeans_10_raw[])
cmeans_10_raw <- as.data.frame(cmeans_10_raw)
cmeans_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_2,Alibates,21
Group_3,Alibates,0
Group_5,Alibates,1
Group_6,Alibates,1
Group_7,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,3


In [176]:
# this one is good, it separates both ESC and most of Oman
# ESC = 210, Oman = 12

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

cmeans_11_raw <- cmeans[cmeans$is_known == "Known",c(10,16)]
cmeans_11_raw <- ftable(cmeans_11_raw[])
cmeans_11_raw <- as.data.frame(cmeans_11_raw)
cmeans_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,0
Group_2,Alibates,21
Group_5,Alibates,0
Group_6,Alibates,2
Group_7,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,4


In [177]:
# this one is good, it separates both ESC and most of Oman
# ESC = 199, Oman = 11

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

cmeans_12_raw <- cmeans[cmeans$is_known == "Known",c(11,16)]
cmeans_12_raw <- ftable(cmeans_12_raw[])
cmeans_12_raw <- as.data.frame(cmeans_12_raw)
cmeans_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,15
Group_5,Alibates,0
Group_6,Alibates,7
Group_7,Alibates,0
Group_8,Alibates,0


In [178]:
# this one is good, it separates both ESC and most of Oman
# ESC = 196, Oman = 12

# Group 1 = 
# Group 2 = 
# Group 3 = 
# Group 4 = 
# Group 5 = 
# Group 6 = 
# Group 7 = 
# Group 8 = 
# Group 9 = 
# Group 10 = 
# Group 11 = 
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

cmeans_13_raw <- cmeans[cmeans$is_known == "Known",c(12,16)]
cmeans_13_raw <- ftable(cmeans_13_raw[])
cmeans_13_raw <- as.data.frame(cmeans_13_raw)
cmeans_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_13,Alibates,0
Group_2,Alibates,18
Group_5,Alibates,0
Group_6,Alibates,5
Group_7,Alibates,0


In [179]:
# this one is good, it separates both ESC and most of Oman
# ESC = 194, Oman = 11

# Group 1 = Chert_3
# Group 2 = Chert_1
# Group 3 = 
# Group 4 = 
# Group 5 = Oman
# Group 6 = Chert_2
# Group 7 = Igneous
# Group 8 = ESC
# Group 9 = 
# Group 10 = 
# Group 11 = ESC
# Group 12 = 
# Group 13 = 
# Group 14 = 
# Group 15 = 

cmeans_14_raw <- cmeans[cmeans$is_known == "Known",c(13,16)]
cmeans_14_raw <- ftable(cmeans_14_raw[])
cmeans_14_raw <- as.data.frame(cmeans_14_raw)
cmeans_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_11,Alibates,0
Group_12,Alibates,0
Group_13,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,7


In [183]:
cmeans_14_art <- cmeans[cmeans$is_known != "Known",c(13,16)]
cmeans_14_art <- ftable(cmeans_14_art[])
cmeans_14_art <- as.data.frame(cmeans_14_art)
cmeans_14_art

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Agate,1
Group_10,Agate,0
Group_11,Agate,0
Group_12,Agate,0
Group_13,Agate,0
Group_14,Agate,0
Group_2,Agate,45
Group_3,Agate,0
Group_4,Agate,0
